In [76]:
!pip install --upgrade pip
!pip install "psycopg2-binary"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.8 MB/s eta 0:00:00


In [71]:
import psycopg2 as pg
import os
from psycopg2 import sql

connection_dict = {"host": "ampfdb.postgres.database.azure.com",
                   "dbname": "machine_data", 
                   "user": "georgeburdell",
                   "password": "Buzz1972",
                   "sslmode": "require",
                   "connect_timeout": 10
                   }

# Construct connection string
conn = pg.connect(**connection_dict)

print("Connection established")

cursor = conn.cursor()

#query = 'GRANT ALL ON SCHEMA public TO waamuser;'

Connection established


In [69]:
cursor.execute(f"DROP TABLE IF EXISTS eos_m280")
conn.commit()

with open(os.path.join('table_strings', 'eos_m280'), 'r') as f:
    query = f.read()
    try:
        cursor.execute(query)
        print("successful transaction")
        conn.commit()
    except:
        print('error')
        conn.rollback()

successful transaction


In [3]:
try: 
    cursor.execute("""SELECT *
                   FROM information_schema.columns
                   WHERE  table_name   = 'eos_m280';
                   """)
    build_schema = []
    for table in cursor.fetchall():
            build_schema.append(table[3])
    conn.commit()
except:
    conn.rollback()

In [72]:
mapping = {'BuildDescription&Purpose': 'build_desc',
 'BuildPlateType': 'build_plate_type',
 'BuildShiftX': 'build_shift_x',
 'BuildShiftY': 'build_shift_y',
 'Customer': 'customer',
 'DatePrinted': 'date_printed',
 'Dosingboost': 'dosing_boost',
 'PrintTime': 'est_print_time',
 'FileLocation': 'build_file_path',
 'GasFlowVoltage': 'gas_flow_voltage',
 'LaserHours': 'laser_hours',
 'MaxCharge': 'max_charge',
 'MinCharge': 'min_charge',
 'Nickname': 'build_nickname',
 'Operator': 'operator',
 'PartHeight': 'part_height',
 'Post-BuildNotes': 'postbuild_notes',
 'PowderHeightUsed': 'used_powder_height',
 'Pre-BuildNotes': 'prebuild_notes',
 'RecoaterType': 'recoater_type',
 'Successful': 'successful',
 'TotalPartVolume': 'total_part_volume',
 'hopper_start': 'hopper_start_level',
 'hoppser_end': 'hopper_end_level',
 'ParameterFileName': 'parameter_set'}


In [105]:
import pandas as pd
import numpy as np
eos = pd.read_excel("eos_builds.xlsx")

eos = eos.rename(columns=mapping)
eos['build_id'] = eos['build_id'].str.lstrip('B').str.lstrip('0').astype(int)
eos = eos.replace(np.nan, "NULL")
eos['powder_lot'] = "NULL"
eos = eos.loc[:, build_schema]
eos.to_csv('eos_builds.csv' , index=False, header=False, sep='|')

In [ ]:
for col in eos.columns:
    if eos[col].dtype == str:
        print(col)
        print(np.any(eos[col].str.contains('|')))

AttributeError: 'Series' object has no attribute 'type'

In [74]:
with open('eos_builds.csv', "r") as file:
    try:
        cursor.copy_from(file, "eos_m280", sep=",", null="None")
        print("Successful Copy")
        conn.commit()
    except:
        conn.rollback()


In [109]:
with open('eos_builds.csv', "r") as file:
    cursor.copy_from(file, "eos_m280", sep='|', null="NULL", columns = eos.columns)

StringDataRightTruncation: value too long for type character varying(50)
CONTEXT:  COPY eos_m280, line 44, column parameter_set: "Aurellio's variable parameter parts. Spinodal lattices for Sanders. Jessie's experiment. ISO artifac..."


In [108]:
conn.rollback()

In [ ]:
cursor.close()
conn.close()